In [ ]:
!pip install import_ipynb
!pip install pyngrok
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

In [23]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
from google.colab import drive
import numpy as np
import keras
import pickle
drive.mount('/gdrive')

import import_ipynb
% cd '/gdrive/MyDrive/Colab Notebooks/'
import data_preprocessing
#% cd '/content'

# Paths
TFIDF_VECTORIZER_PATH = '/gdrive/MyDrive/tfidf_vecctorizer.pkl'
LABEL_ENCODER_PATH = '/gdrive/MyDrive/label_encoder.pkl'
SGD_CLASSIFIER_PATH = '/gdrive/MyDrive/sgd_tfidf'
PADDING_TOKENIZER_PATH = '/gdrive/MyDrive/tokenizer.pkl'
DNN_PATH = '/gdrive/MyDrive/dnn_model'

# Pre-Load Some models and objects
def load_model(path):
  with open(path, 'rb') as f:
    model = pickle.load(f)
  return model

dp = data_preprocessing.DataPreProcessing()
print('Loading vectorizer')
tf_idf_vectorizer = load_model(TFIDF_VECTORIZER_PATH)

print('Loading classifier')
tfidf_sgd_classifier = load_model(SGD_CLASSIFIER_PATH)

print('Loading label encoder')
le = load_model('/gdrive/MyDrive/label_encoder.pkl')

print('Loading padding tokenizer')
padding_tokenizer = load_model(PADDING_TOKENIZER_PATH)

print('Deep Neural Network Model')
dnn_model = keras.models.load_model(DNN_PATH)

! ngrok authtoken 23iNepK2KUiCUd1WBNEtfRcRLN8_4GK3znbNEJKLYnKR8r35e

app = Flask(__name__)
run_with_ngrok(app)

def preprocess_string(sentence):
  refined = dp.preprocess_string(sentence)

@app.route('/')
def my_form():
  return render_template('my-form.html')


@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    processed_text = dp.preprocess_string(text)
    choice = request.form['model']
    prediction = ''
    
    if choice == "Machine learning model":
      transformed_text = tf_idf_vectorizer.transform([processed_text])
      prediction = tfidf_sgd_classifier.predict(transformed_text)
      print(prediction)
      prediction = le.inverse_transform(prediction)
    else:
      padded_text = dp.pad_data([processed_text], 86, padding_tokenizer)
      prediction = dnn_model.predict(padded_text)
      prediction = le.inverse_transform(np.argmax(prediction, axis=1))
      
    return str(prediction[0])

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab Notebooks
Loading vectorizer
Loading classifier
Loading label encoder
Loading padding tokenizer
Deep Neural Network Model
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [24]:
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bffc-35-196-114-111.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Mar/2022 14:03:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:03:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:04:00] "GET /static/img/back.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:04:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:04:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:04:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:04:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:04:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:04:07] "GET /static/img/back.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:04:08] "POST / HTTP/1.1" 200 -


[9]


127.0.0.1 - - [09/Mar/2022 14:04:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:04:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:04:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:04:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:06:43] "POST / HTTP/1.1" 200 -


[3]


127.0.0.1 - - [09/Mar/2022 14:06:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:06:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:06:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:06:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:07:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:07:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:07:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:07:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:07:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:07:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:07:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:07:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:10:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2022 14:10:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2022 14:17:24] "POST / HTTP/1.1" 200 